In [2]:
import torch
import torch.optim as optim

In [3]:
t_c = [0.5,  14.0, 15.0, 28.0, 11.0,  8.0,  3.0, -4.0,  6.0, 13.0, 21.0]
t_u = [35.7, 55.9, 58.2, 81.9, 56.3, 48.9, 33.9, 21.8, 48.4, 60.4, 68.4]
t_c = torch.tensor(t_c).unsqueeze(1)
t_u = torch.tensor(t_u).unsqueeze(1)

In [4]:
n_samples = t_u.shape[0]
n_val = int(0.2 * n_samples)

shuffled_indices = torch.randperm(n_samples)

train_indices = shuffled_indices[:-n_val]
val_indices = shuffled_indices[-n_val:]

In [5]:
t_u_train = t_u[train_indices]
t_c_train = t_c[train_indices]

t_u_val = t_u[val_indices]
t_c_val = t_c[val_indices]

t_un_train = 0.1 * t_u_train
t_un_val = 0.1 * t_u_val

# 6.1

In [1]:
import math
print(math.tanh(-2.2)) # ゴミ収集車
print(math.tanh(0.1)) # クマ
print(math.tanh(2.5)) # イヌ

-0.9757431300314515
0.09966799462495582
0.9866142981514303


# 6.2
## 6.2.1

In [6]:
import torch.nn as nn

linear_model = nn.Linear(1, 1) # コンストラクタの引数を与える
linear_model(t_un_val)

tensor([[-2.0711],
        [-2.7325]], grad_fn=<AddmmBackward0>)

In [7]:
# y = model(x) # このように記述すべき
# y = model.forward(x) # 動くけど危険

## 6.2.2

In [9]:
import torch.nn as nn

# 引数は入力サイズ、出力サイズ、バイアスで、バイアスはデフォルトで True
linear_model = nn.Linear(1, 1)
linear_model(t_un_val)

tensor([[2.7621],
        [3.3204]], grad_fn=<AddmmBackward0>)

In [10]:
linear_model.weight

Parameter containing:
tensor([[0.4652]], requires_grad=True)

In [11]:
linear_model.bias

Parameter containing:
tensor([0.5106], requires_grad=True)

In [12]:
x = torch.ones(1)
linear_model(x)

tensor([0.9758], grad_fn=<AddBackward0>)

In [13]:
x = torch.ones(10, 1)
linear_model(x)

tensor([[0.9758],
        [0.9758],
        [0.9758],
        [0.9758],
        [0.9758],
        [0.9758],
        [0.9758],
        [0.9758],
        [0.9758],
        [0.9758]], grad_fn=<AddmmBackward0>)

In [14]:
t_c = [0.5,  14.0, 15.0, 28.0, 11.0,  8.0,  3.0, -4.0,  6.0, 13.0, 21.0]
t_u = [35.7, 55.9, 58.2, 81.9, 56.3, 48.9, 33.9, 21.8, 48.4, 60.4, 68.4]
t_c = torch.tensor(t_c).unsqueeze(1)
t_u = torch.tensor(t_u).unsqueeze(1)
t_u.shape

torch.Size([11, 1])

In [15]:
linear_model = nn.Linear(1, 1)
optimizer = optim.SGD(linear_model.parameters(), lr=1e-2)

In [16]:
linear_model.parameters()

<generator object Module.parameters at 0x10644e468>

In [17]:
list(linear_model.parameters())

[Parameter containing:
 tensor([[-0.8458]], requires_grad=True),
 Parameter containing:
 tensor([-0.3841], requires_grad=True)]

In [18]:
def training_loop(n_epochs, optimizer, model, loss_fn, t_u_train, t_u_val, t_c_train, t_c_val):
    for epoch in range(1, n_epochs + 1):
        t_p_train = model(t_u_train)
        loss_train = loss_fn(t_p_train, t_c_train)
        
        t_p_val = model(t_u_val)
        loss_val = loss_fn(t_p_val, t_c_val)
        
        optimizer.zero_grad()
        loss_train.backward()
        optimizer.step()
        
        if epoch == 1 or epoch % 1000 == 0:
            print(f"Epoch {epoch}, Training loss {loss_train.item():.4f}, Validation loss {loss_val.item():.4f}")

In [19]:
linear_model = nn.Linear(1, 1)
optimizer = optim.SGD(linear_model.parameters(), lr=1e-2)

training_loop(
n_epochs = 3000,
optimizer = optimizer,
model = linear_model,
loss_fn = nn.MSELoss(), # 手書きで実装していた損失関数の代わり
t_u_train = t_un_train,
t_u_val = t_un_val,
t_c_train = t_c_train,
t_c_val = t_c_val)

print()
print(linear_model.weight)
print(linear_model.bias)

Epoch 1, Training loss 290.6858, Validation loss 169.3326
Epoch 1000, Training loss 2.5371, Validation loss 9.5143
Epoch 2000, Training loss 2.0059, Validation loss 8.8154
Epoch 3000, Training loss 1.9966, Validation loss 8.7306

Parameter containing:
tensor([[5.4040]], requires_grad=True)
Parameter containing:
tensor([-16.9542], requires_grad=True)


誤差の範囲内だとは思う(訓練データとテストデータがランダム分割なので)